# CNN을 이용한 태양광 발전 모델

## data preprocessing

In [4]:
import pandas as pd
import numpy as np
import os
import random
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

np.set_printoptions(precision=6, suppress=True)

df_train = pd.read_csv('train/train.csv')
submission = pd.read_csv('sample_submission.csv')

dfs_test = []

for i in range(81):
    temp_df = pd.read_csv(f"test/{i}.csv")
    dfs_test.append(temp_df.copy())

In [5]:
import math
    
def Make_New_Features(df_train):
    df_train['GHI'] = df_train['DHI'] + df_train['DNI']
    df_train['RDHNI'] = df_train['GHI'].apply(lambda x : x**0.5)
#====================================================================================
    df_train['Lower_Cloud'] = 2.5 * df_train['RH'] / 100 - 1.5
    df_train['Middle_Cloud'] = 4 * df_train['RH'] / 100 - 3.0
#====================================================================================
    b = 17.62
    c = 243.12
    df_train['gamma'] = (b * df_train['T'] /(c + df_train['T'])) + (df_train['RH'] / 100.0).apply(lambda x: math.log(x + 1))
    df_train['DD'] = (c * df_train['gamma']) / (b - df_train['gamma'])

    df_train.drop('gamma', axis=1, inplace=True)
#====================================================================================
        
    return df_train

In [6]:
from sklearn import preprocessing

scalers ={}

def preprocess(df, make_scaler=True):
    df = df.copy()
    df = Make_New_Features(df)
    
    df['Minute'].replace(30, 0.5, inplace=True)
    df['Time'] = df['Hour'] + df['Minute']
    
    df['Time_x'] = np.cos((df['Time'] * 2 * np.pi) / 24.0)
    df['Time_y'] = np.sin((df['Time'] * 2 * np.pi) / 24.0)
    
    scale_target_columns = ['TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                           'Middle_Cloud', 'DD']
    
    temp_df_agg = pd.DataFrame()
    for c in scale_target_columns:
        temp_df_agg[f'max_week_{c}'] = df.groupby('Day')[c].max().rolling(7, center=True).max()
        temp_df_agg[f'min_week_{c}'] = df.groupby('Day')[c].min().rolling(7, center=True).min()

        temp_df_agg[f'max_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'max_week_{c}']
        temp_df_agg[f'max_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'max_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'min_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'min_week_{c}']
        
        
    temp_df_agg = temp_df_agg.reset_index()
    df = pd.merge(df, temp_df_agg, left_on='Day', right_on='Day')
    for c in scale_target_columns:
        df[f'scaled_{c}'] = (df[c] - df[f'min_week_{c}'] + 1) / (df[f'max_week_{c}']+1)

    df['peak_diff'] = df['max_week_TARGET'] - df['TARGET']
    df['peak_diff_scaled'] = df['peak_diff'] / df['max_week_TARGET']
    
    normalize_target_column = ['DHI', 'DNI', 'WS', 'RH', 'T', 'GHI', 'RDHNI', 'TARGET',
                               'Lower_Cloud', 'Middle_Cloud', 'DD']
    
    for c in normalize_target_column:
        x = df[[c]].copy() #returns a numpy array
        if make_scaler:
            std_scaler = preprocessing.MinMaxScaler()
            std_scaler.fit(x)
            df[c] = std_scaler.transform(x)
            scalers[c] = std_scaler
        else:
            df[c] = scalers[c].transform(x)
    
    df = df.drop(['Hour', 'Minute', 'Time'], axis=1)
    
    return df

df = preprocess(df_train)

In [ ]:
temp = []
for df_test in dfs_test:
    temp.append(preprocess(df_test, make_scaler=False))
dfs = temp

In [8]:
def make_train_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    outs_datas = df[['TARGET']].values
    
    ins = []
    outs = []
    
    for i in range(len(df)-48*9):
        ins.append(ins_datas[i:i+48*7])
        outs.append(outs_datas[i+48*8:i+48*9])
        
    return np.array(ins), np.array(outs)

def make_prediction_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    
    ins = [ins_datas]

    return np.array(ins)

ins, outs = make_train_data_set(df)
ins = ins.reshape(ins.shape + (1,))
ins.shape, outs.shape

((52128, 336, 15, 1), (52128, 48, 1))

In [9]:
term = 17376

x_year_1 = ins[:term]
x_year_2 = ins[term:term*2]
x_year_3 = ins[term*2:]

y_year_1 = outs[:term]
y_year_2 = outs[term:term*2]
y_year_3 = outs[term*2:]

drop_columns=[]
x_year_1.shape, x_year_2.shape, x_year_3.shape, y_year_1.shape, y_year_2.shape, y_year_3.shape

((17376, 336, 15, 1),
 (17376, 336, 15, 1),
 (17376, 336, 15, 1),
 (17376, 48, 1),
 (17376, 48, 1),
 (17376, 48, 1))

In [10]:
temp = []
for df in dfs:
    temp.append(make_prediction_data_set(df))
pre_ins = np.concatenate(temp)
pre_ins = pre_ins.reshape(pre_ins.shape + (1, ))
pre_ins.shape

(81, 336, 15, 1)

## Model Preparation

In [11]:
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, GRU, Conv1D, Conv2D
from keras.layers import Flatten, Reshape, Add, MaxPooling2D
from keras.layers import LSTM, Input, Lambda, Concatenate
from keras.layers import RepeatVector, MaxPooling1D, BatchNormalization
from keras.layers import TimeDistributed, Bidirectional, LeakyReLU
from keras import Model
from keras import backend as K
import tensorflow as tf
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


# 7일의 데이터로 2일을 예측
# 1일 = 48개의 관측값
# 7일 = 7 * 48 = 336
# 2일 = 2 * 48 = 96
def pinball_loss(tau):
    def loss(y_true, y_pred):
        err = y_true - y_pred
        return K.mean(K.maximum(tau * err, (tau - 1) * err))
    return loss


def build_model(tau, lr=0.001, use_summary=True, use_mse=False):
    opt = optimizers.Adam(lr=lr)
    
    x_lstm = Input(shape=(336, 15, 1))
    
    x_ins = []
    for i in range(7):
        x_ins.append(Lambda(lambda x: x[:, 48*i:48*(i+1), :, :])(x_lstm))
    
    x_s = []
    for x in x_ins:
        x = Conv2D(20, (1, 4),  padding='same', activation='linear')(x)
        x = Conv2D(20, (3, 1),  padding='same', activation='linear')(x)
        x = Flatten()(x)
        x_s.append(x)
    
    x = Concatenate()(x_s)
    x = Dense(1000)(x)
    x = Dense(100)(x)
    x = Dense(48, activation='relu')(x)

    model = Model(x_lstm, x)
    
    if use_mse:
        model.compile(loss='mse', optimizer=opt)
    else:
        model.compile(loss=pinball_loss(tau), optimizer=opt)
        
    if use_summary:
        model.summary()
    
    return model

In [17]:
from tqdm import tqdm
import numpy as np

def do_dl(q, verbose=0, lr=0.0001):
    es = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True,)
    
    model_1 = build_model(q, lr, use_summary=False)
    his = model_1.fit(np.concatenate([x_year_1, x_year_2]),
                     np.concatenate([y_year_1, y_year_2]),
                     epochs=300, batch_size=2000, validation_data=(x_year_3, y_year_3),
                     verbose=verbose, callbacks=[es])
    model_1.save(f'/content/drive/Shareddrives/Dacon/models/ensemble/model_1_cnn_1_8_{q}_2')

    model_2 = build_model(q, lr, use_summary=False)
    his = model_2.fit(np.concatenate([x_year_1, x_year_3]),
                     np.concatenate([y_year_1, y_year_3]),
                     epochs=300, batch_size=2000, validation_data=(x_year_2, y_year_2),
                     verbose=verbose, callbacks=[es])
    model_2.save(f'/content/drive/Shareddrives/Dacon/models/ensemble/model_2_cnn_1_8_{q}_2')

    model_3 = build_model(q, lr, use_summary=False)
    his = model_3.fit(np.concatenate([x_year_2, x_year_3]),
                     np.concatenate([y_year_2, y_year_3]),
                     epochs=300, batch_size=2000, validation_data=(x_year_1, y_year_1),
                     verbose=verbose, callbacks=[es])
    model_3.save(f'/content/drive/Shareddrives/Dacon/models/ensemble/model_3_cnn_1_8_{q}_2')

In [18]:
do_dl(0.1, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 11s 356ms/step - loss: 0.0585 - val_loss: 0.0177
Epoch 2/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0177 - val_loss: 0.0174
Epoch 3/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0175 - val_loss: 0.0173
Epoch 4/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0173 - val_loss: 0.0172
Epoch 5/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0172 - val_loss: 0.0171
Epoch 6/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0171 - val_loss: 0.0171
Epoch 7/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0171 - val_loss: 0.0170
Epoch 8/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0171 - val_loss: 0.0170
Epoch 9/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0170 - val_loss: 0.0169
Epoch 10/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0170 - val_l

In [19]:
do_dl(0.2, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 336ms/step - loss: 0.0697 - val_loss: 0.0340
Epoch 2/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0337 - val_loss: 0.0326
Epoch 3/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0323 - val_loss: 0.0319
Epoch 4/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0317 - val_loss: 0.0316
Epoch 5/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0316 - val_loss: 0.0315
Epoch 6/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0314 - val_loss: 0.0313
Epoch 7/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0314 - val_loss: 0.0312
Epoch 8/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0311 - val_loss: 0.0311
Epoch 9/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0310 - val_loss: 0.0309
Epoch 10/300
18/18 [==============================] - 6s 326ms/step - loss: 0.0307 - val_lo

In [20]:
do_dl(0.3, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 341ms/step - loss: 0.0720 - val_loss: 0.0464
Epoch 2/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0455 - val_loss: 0.0434
Epoch 3/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0429 - val_loss: 0.0420
Epoch 4/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0416 - val_loss: 0.0409
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0404 - val_loss: 0.0399
Epoch 6/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0395 - val_loss: 0.0394
Epoch 7/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0390 - val_loss: 0.0390
Epoch 8/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0387 - val_loss: 0.0388
Epoch 9/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0385 - val_loss: 0.0386
Epoch 10/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0384 - val_lo

In [21]:
do_dl(0.4, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 335ms/step - loss: 0.0837 - val_loss: 0.0620
Epoch 2/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0602 - val_loss: 0.0565
Epoch 3/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0555 - val_loss: 0.0534
Epoch 4/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0526 - val_loss: 0.0514
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0509 - val_loss: 0.0507
Epoch 6/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0503 - val_loss: 0.0502
Epoch 7/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0497 - val_loss: 0.0498
Epoch 8/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0496 - val_loss: 0.0496
Epoch 9/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0493 - val_loss: 0.0493
Epoch 10/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0491 - val_lo

In [22]:
do_dl(0.5, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 336ms/step - loss: 0.0900 - val_loss: 0.0606
Epoch 2/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0576 - val_loss: 0.0522
Epoch 3/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0505 - val_loss: 0.0482
Epoch 4/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0462 - val_loss: 0.0448
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0432 - val_loss: 0.0434
Epoch 6/300
18/18 [==============================] - 6s 327ms/step - loss: 0.0420 - val_loss: 0.0417
Epoch 7/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0405 - val_loss: 0.0409
Epoch 8/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0397 - val_loss: 0.0403
Epoch 9/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0392 - val_loss: 0.0399
Epoch 10/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0388 - val_lo

In [23]:
do_dl(0.6, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 338ms/step - loss: 0.0968 - val_loss: 0.0708
Epoch 2/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0684 - val_loss: 0.0655
Epoch 3/300
18/18 [==============================] - 6s 315ms/step - loss: 0.0642 - val_loss: 0.0613
Epoch 4/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0594 - val_loss: 0.0584
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0572 - val_loss: 0.0559
Epoch 6/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0548 - val_loss: 0.0541
Epoch 7/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0531 - val_loss: 0.0525
Epoch 8/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0510 - val_loss: 0.0500
Epoch 9/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0492 - val_loss: 0.0493
Epoch 10/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0486 - val_lo

In [24]:
do_dl(0.7, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 354ms/step - loss: 0.0995 - val_loss: 0.0642
Epoch 2/300
18/18 [==============================] - 6s 314ms/step - loss: 0.0601 - val_loss: 0.0518
Epoch 3/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0493 - val_loss: 0.0453
Epoch 4/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0434 - val_loss: 0.0429
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0412 - val_loss: 0.0405
Epoch 6/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0389 - val_loss: 0.0390
Epoch 7/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0377 - val_loss: 0.0381
Epoch 8/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0368 - val_loss: 0.0374
Epoch 9/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0362 - val_loss: 0.0368
Epoch 10/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0358 - val_lo

In [ ]:
do_dl(0.8, verbose=1, lr=.000001)

Epoch 1/300
18/18 [==============================] - 7s 339ms/step - loss: 0.1052 - val_loss: 0.0667
Epoch 2/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0626 - val_loss: 0.0518
Epoch 3/300
18/18 [==============================] - 6s 313ms/step - loss: 0.0493 - val_loss: 0.0455
Epoch 4/300
18/18 [==============================] - 6s 330ms/step - loss: 0.0439 - val_loss: 0.0430
Epoch 5/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0416 - val_loss: 0.0418
Epoch 6/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0407 - val_loss: 0.0409
Epoch 7/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0398 - val_loss: 0.0402
Epoch 8/300
18/18 [==============================] - 6s 310ms/step - loss: 0.0391 - val_loss: 0.0397
Epoch 9/300
18/18 [==============================] - 6s 311ms/step - loss: 0.0389 - val_loss: 0.0392
Epoch 10/300
18/18 [==============================] - 6s 312ms/step - loss: 0.0384 - val_lo

In [ ]:
do_dl(0.9, verbose=1, lr=.000001)

## Model Prediction

In [13]:
import keras

def do_pre_dl(q, day=1):
    with tf.device('/cpu:0'):
        model_1 = keras.models.load_model(f'models/ensemble/model_1_lstm_1_8_{q}_{day}_fined', custom_objects={'loss': pinball_loss(q)})
        model_2 = keras.models.load_model(f'models/ensemble/model_2_lstm_1_8_{q}_{day}_fined', custom_objects={'loss': pinball_loss(q)})
        model_3 = keras.models.load_model(f'models/ensemble/model_3_lstm_1_8_{q}_{day}_fined', custom_objects={'loss': pinball_loss(q)})
    
    pre_1 = model_1.predict(pre_ins)
    pre_2 = model_2.predict(pre_ins)
    pre_3 = model_3.predict(pre_ins)

    pre_1 = scalers['TARGET'].inverse_transform(pre_1)
    pre_2 = scalers['TARGET'].inverse_transform(pre_2)
    pre_3 = scalers['TARGET'].inverse_transform(pre_3)
    
    pre = np.mean([pre_1, pre_2, pre_3], axis=0)

    return pre

In [15]:
import keras

def do_pre_dl(q, day=1):
    with tf.device('/cpu:0'):
        model_1 = keras.models.load_model(f'models/ensemble/model_1_cnn_1_8_{q}_{day}', custom_objects={'loss': pinball_loss(q)})
        model_2 = keras.models.load_model(f'models/ensemble/model_2_cnn_1_8_{q}_{day}', custom_objects={'loss': pinball_loss(q)})
        model_3 = keras.models.load_model(f'models/ensemble/model_3_cnn_1_8_{q}_{day}', custom_objects={'loss': pinball_loss(q)})
    
    pre_1 = model_1.predict(pre_ins)
    pre_2 = model_2.predict(pre_ins)
    pre_3 = model_3.predict(pre_ins)

    pre_1 = scalers['TARGET'].inverse_transform(pre_1)
    pre_2 = scalers['TARGET'].inverse_transform(pre_2)
    pre_3 = scalers['TARGET'].inverse_transform(pre_3)
    
    pre = np.mean([pre_1, pre_2, pre_3], axis=0)

    return pre

In [ ]:
from tqdm import tqdm

submission = pd.read_csv('sample_submission.csv')
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for q in tqdm(taus[7:]):
    pre_1 = do_pre_dl(q, day=1)
    condition = submission['id'].str.contains('Day7')
    submission.loc[condition, "q_"+str(q)] = pre_1.flatten()
    
    pre_2 = do_pre_dl(q, day=2)
    condition = submission['id'].str.contains('Day8')
    submission.loc[condition, "q_"+str(q)] = pre_2.flatten()
    
    submission.to_csv('result/ensemble_lstm_fined.csv', index=False)
    
submission.to_csv('result/ensemble_lstm_fined.csv', index=False)